### Online Transaction History

Online trasacation data is more rich and relevant than in-store purchase data as many transaction are occuring online in response to COViD-19. We will perform and EDA and predict non contractual churn risk of customers.

In [8]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

from lifetimes.utils import *
from lifetimes import BetaGeoFitter,GammaGammaFitter
from lifetimes.plotting import plot_probability_alive_matrix, plot_frequency_recency_matrix
from lifetimes.generate_data import beta_geometric_nbd_model
import matplotlib.pyplot as plt
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases, plot_period_transactions,plot_history_alive



In [9]:
# Bring in data

trnsc = pd.read_csv('transactions_export_1.csv')
orders = pd.read_csv('orders_export_1.csv')

/home/daniel/anaconda3/envs/insight/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (53,56) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
trnsc.head()

,Order,Name,Kind,Gateway,Created At,Status,Amount,Currency,Card Type
0,2518084223139,#0620-13227,sale,paypal,2020-06-12 12:20:47 -0700,success,36.80,USD,NaN
1,2518080782499,#0620-13226,sale,authorize_net,2020-06-12 12:18:25 -0700,success,89.95,USD,visa
2,2518031433891,#0620-13225,sale,authorize_net,2020-06-12 11:51:35 -0700,success,10.50,USD,visa
3,2517958131875,#0620-13224,sale,authorize_net,2020-06-12 11:10:33 -0700,success,72.90,USD,visa
4,2517866184867,#0620-13223,sale,authorize_net,2020-06-12 10:31:55 -0700,success,119.95,USD,master


In [11]:
orders.head()

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,...,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number
0,#0620-13232,missdomine96@gmail.com,paid,2020-06-13 10:31:07 -0700,unfulfilled,NaN,no,USD,110.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,#0620-13231,rreid22@comcast.net,paid,2020-06-13 06:49:15 -0700,unfulfilled,NaN,yes,USD,10.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#0620-13230,penny.com12@gmail.com,paid,2020-06-12 16:47:57 -0700,unfulfilled,NaN,yes,USD,63.30,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,#0620-13230,penny.com12@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#0620-13229,angelanbobby01@aol.com,paid,2020-06-12 16:43:54 -0700,unfulfilled,NaN,yes,USD,36.95,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
trnsc.shape

(16077, 9)

In [13]:
orders.shape

(14462, 72)

I may not actually need the `trnsc` data. I think all I need is contained in the `orders` data.

In [23]:
# Create a unique_id for each customer. Surprisingly it is not in the data.

Email = list(orders.Email.unique())
orders['customer_unique_id'] = orders.Email.apply(lambda x: Email.index(x))

In [24]:
orders.head(10)



,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,...,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,customer_id,customer_unique_id
0,#0620-13232,missdomine96@gmail.com,paid,2020-06-13 10:31:07 -0700,unfulfilled,NaN,no,USD,110.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,#0620-13231,rreid22@comcast.net,paid,2020-06-13 06:49:15 -0700,unfulfilled,NaN,yes,USD,10.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
2,#0620-13230,penny.com12@gmail.com,paid,2020-06-12 16:47:57 -0700,unfulfilled,NaN,yes,USD,63.30,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2
4,#0620-13229,angelanbobby01@aol.com,paid,2020-06-12 16:43:54 -0700,unfulfilled,NaN,yes,USD,36.95,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3
5,#0620-13228,hanksimon@hotmail.com,paid,2020-06-12 15:57:55 -0700,unfulfilled,NaN,no,USD,99.75,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4
6,#0620-13227,rkspillman@gmail.com,paid,2020-06-12 12:20:52 -0700,unfulfilled,NaN,yes,USD,36.80,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,5
7,#0620-13226,annejcourt@sbcglobal.net,paid,2020-06-12 12:18:29 -0700,unfulfilled,NaN,no,USD,89.95,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,6
8,#0620-13225,destinyhamilton911@gmail.com,paid,2020-06-12 11:51:40 -0700,unfulfilled,NaN,no,USD,10.50,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.503348e+10,NaN,7,7
9,#0620-13224,kirsten.roae19@gmail.com,paid,2020-06-12 11:10:36 -0700,unfulfilled,NaN,yes,USD,72.90,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,8
10,#0620-13223,lillywurm@gmail.com,paid,2020-06-12 10:31:58 -0700,unfulfilled,NaN,no,USD,119.95,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,9


#### Building out the dataset

The model only requires a few variables, Let's do some feature engineering and focus only on what we really need for the RFM analysis.

In [25]:
# Distribution of purchases made by customers

orders.groupby('customer_id').size().value_counts()

1     9565
2      748
3      161
4       53
5       17
8        7
7        5
6        5
9        2
19       1
11       1
10       1
80       1
dtype: int64

In [26]:
# Order Id's have multiple entries. Keep the first line as that has price information associated with it.

orders.drop_duplicates('Name',keep='first',inplace=True)

In [27]:
orders.head()

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,...,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,customer_id,customer_unique_id
0,#0620-13232,missdomine96@gmail.com,paid,2020-06-13 10:31:07 -0700,unfulfilled,NaN,no,USD,110.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,#0620-13231,rreid22@comcast.net,paid,2020-06-13 06:49:15 -0700,unfulfilled,NaN,yes,USD,10.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
2,#0620-13230,penny.com12@gmail.com,paid,2020-06-12 16:47:57 -0700,unfulfilled,NaN,yes,USD,63.30,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2
4,#0620-13229,angelanbobby01@aol.com,paid,2020-06-12 16:43:54 -0700,unfulfilled,NaN,yes,USD,36.95,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3
5,#0620-13228,hanksimon@hotmail.com,paid,2020-06-12 15:57:55 -0700,unfulfilled,NaN,no,USD,99.75,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4


In [28]:
orders.to_csv('order_test.csv')

Now subset the data into something more manageable and ditch the datetime stamp

In [29]:
transaction_data= orders[['customer_unique_id','Total','Paid at']]

In [30]:
transaction_data.head()

,customer_unique_id,Total,Paid at
0,0,110.00,2020-06-13 10:31:07 -0700
1,1,10.00,2020-06-13 06:49:15 -0700
2,2,63.30,2020-06-12 16:47:57 -0700
4,3,36.95,2020-06-12 16:43:54 -0700
5,4,99.75,2020-06-12 15:57:55 -0700


In [31]:
transaction_data.columns = transaction_data.columns.str.replace(' ', '_')

In [34]:
list(transaction_data.columns) 

['customer_unique_id', 'Total', 'Paid_at']